<a href="https://colab.research.google.com/github/Dmitriy6655/Library-Python-dly-DS-SEM_8_HW/blob/main/%D0%91%D0%B8%D0%B1%D0%B8%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B0_Python_%D0%B4%D0%BB%D1%8F_DS_%D0%94%D0%97_PyLib_08_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

1. Предобработка данных:  
— Очистите данные от пропусков и аномалий.  
— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

In [ ]:
df = pd.read_csv('/content/Sleep_health_and_lifestyle_dataset.csv')

In [ ]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [ ]:
df.duplicated().sum()

0

In [ ]:
df['Sleep Disorder'].value_counts()

,count
Sleep Disorder,
Sleep Apnea,78
Insomnia,77


Пропусков в датасете фактически нет, но в колонке Sleep Disorder следует заменить NaN на No, тогда она будет отражать суть. К тому же это наша целевая колонка и мы не можем разбивать ее на три колонки.
Кодирования также требуют колонки Gender, Occupation, BMI Category.  
Также потребуется что-то сделать с колонкой Blood Pressure, так как пока она не понятна для модели.

In [ ]:
df['Sleep Disorder'] = df['Sleep Disorder']. fillna ('No')

In [ ]:
# from sklearn. preprocessing import OneHotEncoder

# encoder = OneHotEncoder(handle_unknown='ignore')
# encoder_df = pd.DataFrame(encoder. fit_transform(df[['Sleep Disorder']]). toarray ())
# df = pd.concat([df, encoder_df], axis=1).drop('Sleep Disorder', axis=1)
# df.rename(columns={0: 'Insomnia', 1: 'No_Disorder', 2: 'Sleep Apnea'}, inplace = True )

In [ ]:
df['Sleep_Disorder'] = df['Sleep Disorder'].map({
    'No': 0,
    'Sleep Apnea': 2,
    'Insomnia': 1
})
df = df.drop('Sleep Disorder', axis=1)

In [ ]:
df['Gender'].value_counts()

,count
Gender,
Male,189
Female,185


In [ ]:
df['BMI Category'].value_counts()

,count
BMI Category,
Normal,195
Overweight,148
Normal Weight,21
Obese,10


In [ ]:
df.loc[df['BMI Category'] == 'Normal Weight', 'BMI Category'] = 'Normal'

In [ ]:
from sklearn.preprocessing import LabelEncoder
selected_columns = df.loc[:, ['Gender', 'BMI Category']]
label_encoder = LabelEncoder()

# Кодирование столбцов
encoded_columns = {}
for column in selected_columns:
    df[column + '_encoded'] = label_encoder.fit_transform(df[column])
    # Сохраняем маппинг
    encoded_columns[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Вывод DataFrame
display(df)

# Вывод легенды кодировки для каждого столбца
for column, mapping in encoded_columns.items():
    print(f"Кодировка для {column}:", mapping)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep_Disorder,Gender_encoded,BMI Category_encoded
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,0,1,2
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,0,1,0
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,0,1,0
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,2,1,1
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,2,0,2
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,2,0,2
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,2,0,2
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,2,0,2


Кодировка для Gender: {'Female': 0, 'Male': 1}
Кодировка для BMI Category: {'Normal': 0, 'Obese': 1, 'Overweight': 2}


In [ ]:
df.drop(['Gender', 'BMI Category'], axis = 1, inplace=True)

In [ ]:
df['Occupation'].value_counts()

,count
Occupation,
Nurse,73
Doctor,71
Engineer,63
Lawyer,47
Teacher,40
Accountant,37
Salesperson,32
Software Engineer,4
Scientist,4


In [ ]:
df.loc[df['Occupation'] == 'Sales Representative', 'Occupation'] = 'Salesperson'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование документов в TF-IDF векторы
tfidf_matrix = vectorizer.fit_transform(df['Occupation'])
# Просмотр результатов
print(tfidf_matrix)

# Для получения словаря терминов
print(vectorizer.get_feature_names_out())

  (0, 2)	0.45372742158055335
  (0, 8)	0.8911405202917568
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 6)	1.0
  (4, 6)	1.0
  (5, 2)	0.45372742158055335
  (5, 8)	0.8911405202917568
  (6, 9)	1.0
  (7, 1)	1.0
  (8, 1)	1.0
  (9, 1)	1.0
  (10, 1)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 1)	1.0
  (14, 1)	1.0
  (15, 1)	1.0
  (16, 5)	1.0
  (17, 1)	1.0
  (18, 5)	1.0
  (19, 1)	1.0
  (20, 1)	1.0
  (21, 1)	1.0
  (22, 1)	1.0
  :	:
  (349, 5)	1.0
  (350, 5)	1.0
  (351, 5)	1.0
  (352, 5)	1.0
  (353, 5)	1.0
  (354, 5)	1.0
  (355, 5)	1.0
  (356, 5)	1.0
  (357, 5)	1.0
  (358, 5)	1.0
  (359, 5)	1.0
  (360, 5)	1.0
  (361, 5)	1.0
  (362, 5)	1.0
  (363, 5)	1.0
  (364, 5)	1.0
  (365, 5)	1.0
  (366, 5)	1.0
  (367, 5)	1.0
  (368, 5)	1.0
  (369, 5)	1.0
  (370, 5)	1.0
  (371, 5)	1.0
  (372, 5)	1.0
  (373, 5)	1.0
['accountant' 'doctor' 'engineer' 'lawyer' 'manager' 'nurse' 'salesperson'
 'scientist' 'software' 'teacher']


In [ ]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Конкатенируем исходный DataFrame с новым DataFrame векторов TF-IDF
df = pd.concat([df, tfidf_df], axis=1).drop('Occupation', axis=1)

In [ ]:
df

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Blood Pressure,Heart Rate,Daily Steps,Sleep_Disorder,...,accountant,doctor,engineer,lawyer,manager,nurse,salesperson,scientist,software,teacher
0,1,27,6.1,6,42,6,126/83,77,4200,0,...,0.0,0.0,0.453727,0.0,0.0,0.0,0.0,0.0,0.891141,0.0
1,2,28,6.2,6,60,8,125/80,75,10000,0,...,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,3,28,6.2,6,60,8,125/80,75,10000,0,...,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,4,28,5.9,4,30,8,140/90,85,3000,2,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
4,5,28,5.9,4,30,8,140/90,85,3000,2,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,59,8.1,9,75,3,140/95,68,7000,2,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0
370,371,59,8.0,9,75,3,140/95,68,7000,2,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0
371,372,59,8.1,9,75,3,140/95,68,7000,2,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0
372,373,59,8.1,9,75,3,140/95,68,7000,2,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0


2. Генерация новых признаков:  
— Создайте комбинированные признаки, которые могут помочь улучшить модель.  
— Используйте полиномиальные признаки для числовых переменных.

In [ ]:
df['systolic_pressure'] = df['Blood Pressure'].str.split('/').str[0].astype(int)
df['diastolic_pressure'] = df['Blood Pressure'].str.split('/').str[1].astype(int)

In [ ]:
df = df.drop('Blood Pressure', axis=1)

In [ ]:
df

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep_Disorder,Gender_encoded,...,engineer,lawyer,manager,nurse,salesperson,scientist,software,teacher,systolic_pressure,diastolic_pressure
0,1,27,6.1,6,42,6,77,4200,0,1,...,0.453727,0.0,0.0,0.0,0.0,0.0,0.891141,0.0,126,83
1,2,28,6.2,6,60,8,75,10000,0,1,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,125,80
2,3,28,6.2,6,60,8,75,10000,0,1,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,125,80
3,4,28,5.9,4,30,8,85,3000,2,1,...,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,140,90
4,5,28,5.9,4,30,8,85,3000,2,1,...,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,59,8.1,9,75,3,68,7000,2,0,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,140,95
370,371,59,8.0,9,75,3,68,7000,2,0,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,140,95
371,372,59,8.1,9,75,3,68,7000,2,0,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,140,95
372,373,59,8.1,9,75,3,68,7000,2,0,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,140,95


Я хочу умножить продолжительность сна на его качество, чтобы получить эффективность сна.

In [ ]:
df['Sleep_eff'] = df['Sleep Duration'] * df['Quality of Sleep']

Создадим колонку Lifestyle, которая будет комбинировать уровень физической активности и количество шагов в день.

In [ ]:
# Нормализация данных
scaler = MinMaxScaler()
df_scale = df[['Physical Activity Level', 'Daily Steps']]
df_scale[['Physical Activity Level', 'Daily Steps']] = scaler.fit_transform(df[['Physical Activity Level', 'Daily Steps']])

# Выставление весов
activity_weight = 0.7  # Больший вес, так как зарплата важнее
steps_weight = 0.3  # Меньший вес, так как можно заниматься в зале, плавать и тп. и при этом немного ходить

# Вычисление взвешенной суммы
df['lifestyle'] = df_scale['Physical Activity Level'] * activity_weight + df_scale['Daily Steps'] * steps_weight

df.head()


,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep_Disorder,Gender_encoded,...,manager,nurse,salesperson,scientist,software,teacher,systolic_pressure,diastolic_pressure,Sleep_eff,lifestyle
0,1,27,6.1,6,42,6,77,4200,0,1,...,0.0,0.0,0.0,0.0,0.891141,0.0,126,83,36.6,0.191429
1,2,28,6.2,6,60,8,75,10000,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,125,80,37.2,0.650000
2,3,28,6.2,6,60,8,75,10000,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,125,80,37.2,0.650000
3,4,28,5.9,4,30,8,85,3000,2,1,...,0.0,0.0,1.0,0.0,0.000000,0.0,140,90,23.6,0.000000
4,5,28,5.9,4,30,8,85,3000,2,1,...,0.0,0.0,1.0,0.0,0.000000,0.0,140,90,23.6,0.000000


3. Отбор признаков:  
— Примените несколько методов отбора признаков (например, RFE, SelectKBest).  
— Сравните качество модели до и после отбора признаков.

In [ ]:
X = df.drop('Sleep_Disorder', axis=1)
y = df['Sleep_Disorder']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
tree = DecisionTreeRegressor()

tree.fit(X_train, y_train)
prediction = tree.predict(X_test)

print(f'R^2 на тренировочных данных {r2_score(y_train, tree.predict(X_train))}')
print(f'R^2 на тестовых данных {r2_score(y_test, tree.predict(X_test))}')

R^2 на тренировочных данных 1.0
R^2 на тестовых данных 0.8208786231884058


С настройками по умолчанию модель кажется переобученной, поскольку мы получаем 1 на тренировочных данных. При этом на тестовых данных коэффициент детерминации остается достаточно высоким.

In [ ]:
from sklearn.feature_selection import RFE
# Создаем RFE модель
rfe = RFE(estimator=tree, n_features_to_select=13)
rfe = rfe.fit(X, y)

# Показываем какие признаки выбраны
selected_features = pd.DataFrame({
    'Feature': X.columns,  # Здесь X.columns должен содержать названия столбцов, использованных для X
    'Ranking': rfe.ranking_
})
print(selected_features.sort_values(by='Ranking', ascending=False))

                    Feature  Ranking
23                lifestyle       12
18                 software       11
10               accountant       10
20        systolic_pressure        9
8            Gender_encoded        8
17                scientist        7
6                Heart Rate        6
16              salesperson        5
3          Quality of Sleep        4
13                   lawyer        3
4   Physical Activity Level        2
15                    nurse        1
22                Sleep_eff        1
21       diastolic_pressure        1
19                  teacher        1
0                 Person ID        1
14                  manager        1
1                       Age        1
11                   doctor        1
9      BMI Category_encoded        1
7               Daily Steps        1
5              Stress Level        1
2            Sleep Duration        1
12                 engineer        1


In [ ]:
top_features = selected_features.sort_values(by='Ranking', ascending=False)[:12]
X_train_selected = X_train[top_features.Feature]
X_test_selected = X_test[top_features.Feature]

# Обучение дерева решений на отобранных признаках
dt1 = DecisionTreeRegressor(random_state=42)
dt1.fit(X_train_selected, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, dt1.predict(X_train_selected))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt1.predict(X_test_selected))}')

R^2 на тренировочных данных 0.7828517177847449
R^2 на тестовых данных 0.750660169320012


Если обучать модель на топ 12 признаках (половине), то мы как будто бы избавляемся от переобучения. К тому же, разница между коэффициентом детерминации на тестовых и тренировочных данных сокращается.

In [ ]:
feature_importances = pd.Series(tree.feature_importances_, index=X_train.columns)

# Отбор 12 наиболее важных признаков
top_features2 = feature_importances.nlargest(12).index
top_features2

Index(['BMI Category_encoded', 'Person ID', 'systolic_pressure', 'Age',
       'Sleep_eff', 'Heart Rate', 'Gender_encoded', 'Sleep Duration',
       'Quality of Sleep', 'teacher', 'lawyer', 'Physical Activity Level'],
      dtype='object')

В этом случае мы получили другой набор признаков.

In [ ]:
X_train_selected2 = X_train[top_features2]
X_test_selected2 = X_test[top_features2]

# Обучение дерева решений на отобранных признаках
dt2 = DecisionTreeRegressor(random_state=42)
dt2.fit(X_train_selected2, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, dt2.predict(X_train_selected2))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt2.predict(X_test_selected2))}')

R^2 на тренировочных данных 1.0
R^2 на тестовых данных 0.8208786231884058


Результаты лучше, но 1 на тренировочных данных.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd

# Предположим, что X - ваши признаки, а y - целевая переменная.
# Создаем SelectKBest модель с выбранной функцией оценки
select_k_best = SelectKBest(score_func=f_classif, k=12)  # k - количество признаков, которые нужно выбрать
select_k_best.fit(X, y)

# Получаем оценки признаков
scores = select_k_best.scores_

# Создаем DataFrame для выбранных признаков и их оценок
selected_features3 = pd.DataFrame({
    'Feature': X.columns,
    'Score': scores
})

# Сортируем по убыванию оценок
print(selected_features3.sort_values(by='Score', ascending=False))


                    Feature       Score
9      BMI Category_encoded  308.683826
21       diastolic_pressure  268.097906
15                    nurse  253.806586
20        systolic_pressure  214.585414
16              salesperson   64.394013
1                       Age   58.408963
0                 Person ID   54.653439
4   Physical Activity Level   44.150579
23                lifestyle   42.785715
19                  teacher   35.694103
6                Heart Rate   32.948639
2            Sleep Duration   31.913468
8            Gender_encoded   31.510655
22                Sleep_eff   30.914861
3          Quality of Sleep   27.600604
7               Daily Steps   24.776923
11                   doctor   19.796018
12                 engineer   18.286549
13                   lawyer   11.077560
5              Stress Level    6.603565
10               accountant    6.243088
17                scientist    1.263244
18                 software    0.531321
14                  manager    0.352653


In [ ]:
top_features3 = selected_features3.sort_values(by='Score', ascending=False)[:12]
X_train_selected3 = X_train[top_features3.Feature]
X_test_selected3 = X_test[top_features3.Feature]

# Обучение дерева решений на отобранных признаках
dt3 = DecisionTreeRegressor(random_state=42)
dt3.fit(X_train_selected3, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, dt3.predict(X_train_selected3))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt3.predict(X_test_selected3))}')

R^2 на тренировочных данных 1.0
R^2 на тестовых данных 0.8208786231884058


В целом, с уменьшением признаков качество модели снижается. Но во всяком случае мы не получаем переобученную модель при RFE.

4. Подбор гиперпараметров:  
— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.  
— Оцените, как изменение гиперпараметров влияет на качество предсказаний.

In [ ]:
from sklearn.model_selection import GridSearchCV
# Функция для оценки модели с помощью перекрестной проверки
def evaluate_model(model, X_train, y_train):
    scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=5)
    return scores.mean()

# Поиск гиперпараметров с помощью сетки параметров
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 6]
}

grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

print(f"Лучшие гиперпараметры для модели из коробки (Grid Search): {grid_search.best_params_}")
print(f"Лучшая оценка для модели из коробки (Grid Search): {grid_search.best_score_:.2f}")

Лучшие гиперпараметры для модели из коробки (Grid Search): {'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 5}
Лучшая оценка для модели из коробки (Grid Search): 0.68


In [ ]:
# Функция для оценки модели с помощью перекрестной проверки
def evaluate_model(model, X_train, y_train):
    scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=5)
    return scores.mean()

# Поиск гиперпараметров с помощью сетки параметров
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 6]
}

grid_search = GridSearchCV(estimator=dt1, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

print(f"Лучшие гиперпараметры для dt1 (Grid Search): {grid_search.best_params_}")
print(f"Лучшая оценка для dt1 (Grid Search): {grid_search.best_score_:.2f}")

Лучшие гиперпараметры для dt1 (Grid Search): {'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 2}
Лучшая оценка для dt1 (Grid Search): 0.71


При кросс-валидации подтвердилось, что наша модель из коробки все же несколько переобучена, так как мы получили существенно более низкое значение 0.68.  
Модель с меньшим числом признаков кажется даже более эффективной.  
При этом для модели с меньшим числом признаков Grid Search выбрал большую глубину дерева и большее количество данных в листе, за счет чего, вероятно, и увеличил ее способность к обобщению, чтобы получить наилучший результат.